# BANK MARKETING

<br><br>
Membros:
- Anderson Jesus
- Caio Viera
- Pedro Correia



> CRIAÇÃO DE MODELO PREDITIVO

#### Inicializando sessão do Spark

In [1]:
import findspark
findspark.init('/home/pfcor/spark-2.1.0-bin-hadoop2.7')

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName('bank').getOrCreate()

#### Carregando os Dados

In [3]:
# data = spark.read.csv(
#     "hdfs://elephant:8020/user/labdata/bank.csv",
#     header=True,
#     sep=";",
#     inferSchema=True
# )

In [7]:
data = spark.read.csv(
    'data/historical-data.csv',
    sep=';',
    header=True,
    inferSchema=True
)

In [8]:
data = data.selectExpr(*["`{}` as {}".format(col, col.replace('.', '_')) for col in data.columns])

#### Preparação dos Dados

Definindo variáveis utilizadas pelo tipo a fim de realizar o encoding necessário

In [9]:
categoricalColumns = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'month',
    'day_of_week',
    'poutcome'
]

# não utilizaremos a variável `duration`, que algo não sabido antes da ligação ocorrer
# e portanto, não deve ser válida para fins preditivos
numericColumns = [
    'pdays',
    'previous',
    'emp_var_rate',
    'cons_price_idx',
    'cons_conf_idx',
    'euribor3m',
    'nr_employed'
]

Iniciando a construção do Pipeline

In [10]:
from pyspark.ml import Pipeline

In [11]:
# encoders e indexadores necessários ao tratamento dos dados
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

# instanciando nossa lista de passos a serem fornecidos ao pipeline
stages = []

Target

In [12]:
# indexação da variável resposta
indexer = StringIndexer(
    inputCol='y', 
    outputCol='label'
)

stages += [indexer]

Dados Categóricos

In [13]:
# transformações dados categóricos (paralelo pandas: get_dummies)
for categoricalCol in categoricalColumns:
    # nomes para valores [0:n_cats-1]
    indexer = StringIndexer(
        inputCol=categoricalCol, 
        outputCol=categoricalCol+'_index'
    )
    # criando dummies
    encoder = OneHotEncoder(
        inputCol=categoricalCol+'_index',
        outputCol=categoricalCol+'_class_vec'
    )
    # inserindo estágios de transformação
    stages += [indexer, encoder]

Dados Numéricos

In [14]:
# transformando variáveis numéricas para o tipo double
for numericCol in numericColumns:
    data = data.withColumn(numericCol, data[numericCol].cast('double'))

Assembler

In [15]:
# criando assembler, que deixa os dados no formato vetorial 
# demandado pela biblioteca ML do Spark

assembler_inputs = [categoricalCol+'_class_vec' for categoricalCol in categoricalColumns]
assembler_inputs += numericColumns
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

stages += [assembler]

Criação do Pipeline finalizada e fit/transform realizado na base

In [18]:
# instanciando Pipeline com os estágios criados acima
pipeline = Pipeline(stages=stages)

# fit na base
pipelineModel = pipeline.fit(data)

# salvando o pipeline
pipelineModel.write().overwrite().save('model/bank-pipeline')

In [19]:
# transformando enfim os dados para o treinar o modelo
data_model = pipelineModel.transform(data)
data_model = data_model.select(["label", "features"]) # reduzindo a quantidade de dados a serem processados

#### Modelagem

*Gradient Boosting Machine*

Criando bases de treino e teste

In [20]:
# separação de dados em treino e teste
(trainingData, testData) = data_model.randomSplit([0.8, 0.2], seed=420)

print('Observações para treino: {}'.format(trainingData.count()))
print('Observações para teste:  {}'.format(testData.count()))

Observações para treino: 29705
Observações para teste:  7364


Iniciando modelagem propriamente dita (hiperparâmetros baseados em exploratória realizada anteriormente)

In [21]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(
    labelCol="label",
    featuresCol="features",
    predictionCol='prediction',
    maxIter=60,
    stepSize=0.01,
    seed=420
)

In [45]:
%%time
gbtModel = gbt.fit(trainingData)

# salvando o modelo
gbtModel.write().overwrite().save('model/bank-gbtmodel')

CPU times: user 16 ms, sys: 1.71 ms, total: 17.7 ms
Wall time: 1min 5s


In [24]:
print(gbtModel)

GBTClassificationModel (uid=GBTClassifier_44f8833c837bf6bc4c35) with 60 trees


Realizando predições

In [25]:
predictions_gbt = gbtModel.transform(testData)
predictions_gbt_train = gbtModel.transform(trainingData)

Verificando qualidade das predições

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [27]:
evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction",
    metricName="accuracy"
)

# evaluator_auc = BinaryClassificationEvaluator(
#     labelCol="label", 
#     rawPredictionCol="rawPrediction"
# )

accuracy_gbt = evaluator_accuracy.evaluate(predictions_gbt)
accuracy_gbt_train = evaluator_accuracy.evaluate(predictions_gbt_train)
print('Accuracy:         {:.4f}'.format(accuracy_gbt))
print('Accuracy (TRAIN): {:.4f}'.format(accuracy_gbt_train))
# auc_gbt = evaluator_auc.evaluate(predictions_gbt)
# print('areaUnderROC:     {:.4f}'.format(auc_gbt))

Accuracy:         0.9005
Accuracy (TRAIN): 0.9054


In [54]:
predictions_gbt.select('label', 'prediction').createOrReplaceTempView('predictions')

spark.sql("""
SELECT
    round((tp+tn)/(tp+tn+fp+fn), 4) as accuracy,
    round(tp/(tp+fp), 4) as precision,
    round(tp/(tp+fn), 4) as recall
FROM (
    SELECT
        sum(tn) as tn,
        sum(tp) as tp,
        sum(fn) as fn,
        sum(fp) as fp
    FROM (
        SELECT
            case when label = 0 and prediction = 0 then 1 else 0 end as tn,
            case when label = 1 and prediction = 1 then 1 else 0 end as tp,
            case when label = 1 and prediction = 0 then 1 else 0 end as fn,
            case when label = 0 and prediction = 1 then 1 else 0 end as fp
        FROM
            predictions
    )
)
""").show()

+--------+---------+------+
|accuracy|precision|recall|
+--------+---------+------+
|  0.9005|   0.6369|0.2518|
+--------+---------+------+



#### Conclusão

`GBTClassifier` apresentou maior potencial, apesar de ter um recall ainda muito baixo. 

Na sequência, vamos buscar realizar um resampling da base, buscando deixar a classe positiva mais prevalente.

In [60]:
(data_res, holdOutData) = data_model_res.randomSplit([0.8, 0.2], seed=420)

In [61]:
data_res = data_res.sampleBy('y', fractions={'yes': 0.8, 'no': 0.2})

In [62]:
data_res.groupBy('y').count().show()

+---+-----+
|  y|count|
+---+-----+
| no| 1050|
|yes| 2069|
+---+-----+



Como podemos ver, a quantidade de dados utilizados diminuiu bastante. Assim, esperamos uma redução na acurácia do modelo, mas a espectativa é que o recall melhore, o que é o que nós buscamos nesse momento.

In [58]:
# # preprocessamento
# data_model_res = pipelineModel.transform(data_res)

# (trainingData_res, testData_res) = data_model_res.randomSplit([0.8, 0.2], seed=420)
# print('Observações para treino: {}'.format(trainingData_res.count()))
# print('Observações para teste:  {}'.format(testData_res.count()))

In [63]:
# criando novo modelo com resampling
gbtModel_res = gbt.fit(data_res)

# salvando o modelo
gbtModel.write().overwrite().save('model/bank-gbtmodel-res')

In [50]:
predictions_gbt_res = gbtModel_res.transform(testData_res)

In [51]:
accuracy_gbt_res = evaluator_accuracy.evaluate(predictions_gbt_res)
print('Accuracy:         {:.4f}'.format(accuracy_gbt_res))

Accuracy:         0.7923


In [53]:
predictions_gbt_res.select('label', 'prediction').createOrReplaceTempView('predictions_res')

spark.sql("""
SELECT
    round((tp+tn)/(tp+tn+fp+fn), 4) as accuracy,
    round(tp/(tp+fp), 4) as precision,
    round(tp/(tp+fn), 4) as recall
FROM (
    SELECT
        sum(tn) as tn,
        sum(tp) as tp,
        sum(fn) as fn,
        sum(fp) as fp
    FROM (
        SELECT
            case when label = 0 and prediction = 0 then 1 else 0 end as tn,
            case when label = 1 and prediction = 1 then 1 else 0 end as tp,
            case when label = 1 and prediction = 0 then 1 else 0 end as fn,
            case when label = 0 and prediction = 1 then 1 else 0 end as fp
        FROM
            predictions_res
    )
)
""").show()

+--------+---------+------+
|accuracy|precision|recall|
+--------+---------+------+
|  0.7923|   0.7661|0.5663|
+--------+---------+------+



Como esperávamos, o recall apresentou um valor bem mais interessante, demonstrando o potencial da solução, que certamente poderia ser refinada ainda mais a fim de atingir resultados mais expressivos.